In [ ]:

import sys
sys.path.insert(0, '../src/')

from loopless_utils import loops_enumeration_from_fva

from load_modify_sample_utils import load_model, get_objective_functions, get_reaction_bounds, modify_model
from load_modify_sample_utils import sample_optgp

from correlations_utils import correlated_reactions, split_forward_reverse
from correlations_utils import find_reactants_products

from graphs_utils import construct_graph, plot_graph, compute_nodes_centrality_metrics
from graphs_utils import compare_network_modularities, compare_node_centralities, compare_essential_to_network_central_reactions

from pathways_utils import dictionary_reaction_id_to_kegg_id, get_kegg_pathways_from_reaction_ids
from pathways_utils import map_model_to_kegg_reactions_dictionary, read_json_file, fill_missing_kegg_ids_in_initial_dictionary
from pathways_utils import subset_model_reactions_from_pathway_info, subset_sampling_array_from_reaction_ids, sort_reactions_by_model_order
from pathways_utils import dictionary_reaction_id_to_pathway, dictionary_map_reverse_reaction_id_to_pathway


In [ ]:

ec_cobra_model, ec_cobra_reactions, ec_cobra_reaction_ids,  = load_model("../ext_data/models/e_coli_core.xml")

objective_functions = get_objective_functions(ec_cobra_model)
print(objective_functions)

default_reaction_bounds = get_reaction_bounds(ec_cobra_model)
print(default_reaction_bounds)


In [ ]:

# Set optimal percentage to 100
ec_cobra_model_condition_100 = modify_model(
    cobra_model         = ec_cobra_model,
    objective_function  = "BIOMASS_Ecoli_core_w_GAM",
    optimal_percentage  = 100,
    objective_direction = "max"
)

updated_objective_functions = get_objective_functions(ec_cobra_model_condition_100)
print(updated_objective_functions)

updated_reaction_bounds = get_reaction_bounds(ec_cobra_model_condition_100)
print(updated_reaction_bounds.get("BIOMASS_Ecoli_core_w_GAM"))

# -----------

# Set optimal percentage to 0
ec_cobra_model_condition_0 = modify_model(
    cobra_model         = ec_cobra_model,
    objective_function  = "BIOMASS_Ecoli_core_w_GAM",
    optimal_percentage  = 0,
    objective_direction = "max"
)

updated_objective_functions = get_objective_functions(ec_cobra_model_condition_0)
print(updated_objective_functions)

updated_reaction_bounds = get_reaction_bounds(ec_cobra_model_condition_0)
print(updated_reaction_bounds.get("BIOMASS_Ecoli_core_w_GAM"))

In [ ]:

loopy_reactions_fva_100 = loops_enumeration_from_fva(ec_cobra_model_condition_100, fraction_of_optimum=0.999)
print(loopy_reactions_fva_100)

loopy_reactions_100 = [item[0] for item in loopy_reactions_fva_100]
print(loopy_reactions_100)


loopy_reactions_fva_0 = loops_enumeration_from_fva(ec_cobra_model_condition_0, fraction_of_optimum=0)
print(loopy_reactions_fva_0)

loopy_reactions_0 = [item[0] for item in loopy_reactions_fva_0]
print(loopy_reactions_0)


In [ ]:

ec_cobra_model_condition_100.reactions.get_by_id("FRD7").bounds = (0, 0)
ec_cobra_model_condition_0.reactions.get_by_id("FRD7").bounds = (0, 0)

In [ ]:

samples_optgp_condition_100 = sample_optgp(ec_cobra_model_condition_100, 
                                           n_samples = 3000, 
                                           thinning=100, 
                                           reaction_in_rows = True)

samples_optgp_condition_0 = sample_optgp(ec_cobra_model_condition_0, 
                                         n_samples = 3000, 
                                         thinning=100, 
                                         reaction_in_rows = True)

In [ ]:

initial_bigg_to_kegg_dictionary = map_model_to_kegg_reactions_dictionary(ec_cobra_model)

reactions_json, reactions_pandas = read_json_file("../ext_data/reactions/reactions.json")

bigg_to_kegg, seed_to_kegg = dictionary_reaction_id_to_kegg_id(reactions_pandas)

final_bigg_to_kegg_dictionary = fill_missing_kegg_ids_in_initial_dictionary(initial_bigg_to_kegg_dictionary, 
                                                                            modeltype="BiGG", 
                                                                            bigg_to_kegg=bigg_to_kegg,
                                                                            seed_to_kegg=seed_to_kegg)

df_kegg_pathways = get_kegg_pathways_from_reaction_ids(final_bigg_to_kegg_dictionary)


In [ ]:

extended_steady_states_100_full, extended_reactions_100 = split_forward_reverse(samples_optgp_condition_100, reactions=ec_cobra_reaction_ids)

reversibility_list_all_reactions_full, reactants_list_all_reactions_full, products_list_all_reactions_full = find_reactants_products(ec_cobra_model, extended_reactions_100)
print(len(reversibility_list_all_reactions_full), len(reactants_list_all_reactions_full))


extended_steady_states_0_full, extended_reactions_0 = split_forward_reverse(samples_optgp_condition_0, reactions=ec_cobra_reaction_ids)

reversibility_list_all_reactions_full, reactants_list_all_reactions_full, products_list_all_reactions_full = find_reactants_products(ec_cobra_model, extended_reactions_0)
print(len(reversibility_list_all_reactions_full), len(reactants_list_all_reactions_full))


In [ ]:

(linear_correlation_matrix_100_full, 
non_linear_correlation_matrix_100_full, 
mixed_correlation_matrix_100_full, 
correlations_dictionary_100_full) = correlated_reactions(
        steady_states = extended_steady_states_100_full,
        boolean_sharing_metabolites_matrix=None,
        reactions=extended_reactions_100,
        linear_coeff = "pearson", 
        linear_corr_cutoff = 0.3, 
        indicator_cutoff = 1.2,
        jensenshannon_cutoff = 0.05,
        std_cutoff= 1e-2,
        include_non_linear = True, 
        cells = 5, 
        cop_coeff = 0.2, 
        lower_triangle = False, 
        verbose = True)



(linear_correlation_matrix_0_full, 
non_linear_correlation_matrix_0_full, 
mixed_correlation_matrix_0_full, 
correlations_dictionary_0_full) = correlated_reactions(
        steady_states = extended_steady_states_0_full,
        boolean_sharing_metabolites_matrix=None,
        reactions=extended_reactions_0,
        linear_coeff = "pearson", 
        linear_corr_cutoff = 0.3, 
        indicator_cutoff = 1.2,
        jensenshannon_cutoff = 0.05,
        std_cutoff= 1e-2,
        include_non_linear = True, 
        cells = 5, 
        cop_coeff = 0.2, 
        lower_triangle = False, 
        verbose = True)

In [ ]:

Glycolysis = subset_model_reactions_from_pathway_info(df_kegg_pathways, "Glycolysis / Gluconeogenesis")
PPP = subset_model_reactions_from_pathway_info(df_kegg_pathways, "Pentose phosphate pathway")

bigg_to_pathway_dict = dictionary_reaction_id_to_pathway(Glycolysis = Glycolysis, PPP = PPP)

group_map_100_full = dictionary_map_reverse_reaction_id_to_pathway(bigg_to_pathway_dict, extended_reactions_100)
group_map_0_full = dictionary_map_reverse_reaction_id_to_pathway(bigg_to_pathway_dict, extended_reactions_0)


In [ ]:

G100_full, pos100_full = construct_graph(linear_correlation_matrix_100_full,
                         non_linear_correlation_matrix_100_full,
                         reactions=extended_reactions_100,
                         remove_unconnected_nodes=False,
                         correction=False,
                         group_map=group_map_100_full)


G0_full, pos0_full = construct_graph(linear_correlation_matrix_0_full,
                         non_linear_correlation_matrix_0_full,
                         reactions=extended_reactions_0,
                         remove_unconnected_nodes=False,
                         correction=False,
                         group_map=group_map_0_full)


In [ ]:

centrality_dict_100 = compute_nodes_centrality_metrics(G100_full)

centrality_dict_0 = compute_nodes_centrality_metrics(G0_full)

### Find how many of the reactions with a high network centrality metric belong to the essential reactions

In [ ]:

central_and_essential_reactions, essential_reactions_count, matched_reactions = compare_essential_to_network_central_reactions(
                                                                                ec_cobra_model,
                                                                                centrality_dict_100.get("degree"),
                                                                                threshold=0.999)
print(central_and_essential_reactions, essential_reactions_count)



central_and_essential_reactions, essential_reactions_count, matched_reactions = compare_essential_to_network_central_reactions(
                                                                                ec_cobra_model,
                                                                                centrality_dict_100.get("betweenness"),
                                                                                threshold=0.999)
print(central_and_essential_reactions, essential_reactions_count)



central_and_essential_reactions, essential_reactions_count, matched_reactions = compare_essential_to_network_central_reactions(
                                                                                ec_cobra_model,
                                                                                centrality_dict_100.get("clustering"),
                                                                                threshold=0.999)
print(central_and_essential_reactions, essential_reactions_count)


### Compare centrality metrics with flux sampling metrics